Testing parallel SA algorithm in Josiann

In [1]:
# imports
import numpy as np
import plotly.graph_objects as go

import josiann as jo
from josiann.moves.parallel.set import ParallelSetStep

/home/mbouvier/git/josiann/josiann/sequential/base/sa.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def cost(x: np.ndarray, n: np.ndarray) -> np.ndarray:
    return 0.6 + np.sum(
        np.sin(1 - 16 / 15 * x) ** (n + 1)
        - 1 / 50 * np.sin(4 - 64 / 15 * x) ** n
        - np.sin(1 - 16 / 15 * x) ** n
    )

In [3]:
nx, ny = 100, 100

X = np.linspace(-1, 1, nx)
Y = np.linspace(-1, 1, ny)

xv, yv = np.meshgrid(X, Y)
coords = np.concatenate((xv.flatten()[:, None], yv.flatten()[:, None]), axis=1)

In [ ]:
fig = go.Figure(data=[go.Surface(x=X, y=Y, z=np.array([cost(c, 0) for c in coords]).reshape(nx, ny))])
fig.show()

In [5]:
fig = go.Figure(data=[go.Surface(x=X, y=Y, z=np.array([cost(c, 1) for c in coords]).reshape(nx, ny))])
fig.show()

In [6]:
fig = go.Figure(data=[go.Surface(x=X, y=Y, z=np.array([cost(c, 2) for c in coords]).reshape(nx, ny))])
fig.show()

In [ ]:
# ---------------

In [7]:
# vectorized cost function
def vect_cost(x: np.ndarray, _converged: np.ndarray, n: np.ndarray) -> np.ndarray:
    print(_converged, x)
    return 0.6 + np.sum(
        [
            np.sin(1 - 16 / 15 * x[:, i]) ** (n + 1)
            - 1 / 50 * np.sin(4 - 64 / 15 * x[:, i]) ** n
            - np.sin(1 - 16 / 15 * x[:, i]) ** n
            for i in range(x.shape[1])
        ],
        axis=0,
    )

In [8]:
res = jo.psa(vect_cost,
             np.array([[0, 0], [0, 0], [0, 0]]),
             parallel_args=[np.array([0, 1, 2])],
             bounds=[(-1, 1), (-1, 1)],
             moves=ParallelSetStep(np.tile(np.arange(-1, 1.1, 0.1), (2, 1))),
             max_measures=1,
             max_iter=2000,
             backup=True)

[False False False] [[0. 0.]
 [0. 0.]
 [0. 0.]]


  0%|          | 0/2000 [00:00<?, ?iteration/s]

[False False False] [[ 1.00000000e-01  0.00000000e+00]
 [ 1.00000000e-01  0.00000000e+00]
 [-2.22044605e-16  0.00000000e+00]]
[False False False] [[ 1.00000000e-01 -2.22044605e-16]
 [ 1.00000000e-01  1.00000000e-01]
 [-2.22044605e-16 -2.22044605e-16]]
[False False False] [[ 2.00000000e-01 -2.22044605e-16]
 [-2.22044605e-16  1.00000000e-01]
 [ 1.00000000e-01 -2.22044605e-16]]
[False False False] [[ 2.00000000e-01  1.00000000e-01]
 [-2.22044605e-16  2.00000000e-01]
 [ 1.00000000e-01 -1.00000000e-01]]
[False False False] [[ 1.00000000e-01  1.00000000e-01]
 [ 1.00000000e-01  2.00000000e-01]
 [-2.22044605e-16 -1.00000000e-01]]
[False False False] [[ 1.00000000e-01  2.00000000e-01]
 [ 1.00000000e-01  1.00000000e-01]
 [-2.22044605e-16 -2.22044605e-16]]
[False False False] [[-2.22044605e-16  2.00000000e-01]
 [ 2.00000000e-01  1.00000000e-01]
 [ 1.00000000e-01 -2.22044605e-16]]
[False False False] [[-2.22044605e-16  1.00000000e-01]
 [ 2.00000000e-01  2.00000000e-01]
 [ 1.00000000e-01 -1.0000000

In [21]:
res

Result(message='Convergence tolerance reached.', success=True, trace=Parallel trace of 1584 iteration(s), 3 parallel problem(s) and 2 dimension(s)., parameters=ParallelSAParameters(multi=MultiParameters(nb_walkers=3), moves=	ParallelSetStep with probability 1.0
, fun=<function vect_cost at 0x7f4ffd2be550>, backup=Backup: active, costs=[nan, -1.573234589846785, nan], last_ns=[1, 1, 1], window_size=200, seed=1677591343, base=ParallelBaseParameters(args=(), x0=array([[0., 0.],
       [0., 0.],
       [0., 0.]]), max_iter=2000, max_measures=1, final_acceptance_probability=1e-300, epsilon=0.01, T_0=5.0, tol=0.001, alpha=0.9959346595447985, sigma_max=0.0, suppress_warnings=False, detect_convergence=True, parallel_args=[array([0, 1, 2])])))

In [19]:
res.trace.plot_positions(extended=True, true_values=np.array([[1, 1], [0.47, 0.47], [0.31, 0.31]]))

In [28]:
vect_cost(np.array([[0.3, 0.3], [0.4, 0.3], [0.5, 0.4], [0.5, 0.5]]), n=0)

array([-0.18241395, -0.26877159, -0.44765274, -0.54017624])

In [25]:
cost(np.array([0.3, 0.3]), n=0)

-0.1824139519630631

In [23]:
def vect_cost2(x: np.ndarray,
              n: np.ndarray) -> np.ndarray:
    return 0.6 + np.sum([np.sin(1 - 16 / 15 * x[:, i]) ** (n+1) -
                         1 / 50 * np.sin(4 - 64 / 15 * x[:, i]) ** n -
                         np.sin(1 - 16 / 15 * x[:, i]) ** n
                         for i in range(x.shape[1])], axis=0)

In [24]:
vect_cost2(np.array([[0.3, 0.3], [0.4, 0.3], [0.5, 0.4], [0.5, 0.5]]), n=np.array([1, 1, 1, 1]))


array([0.11680672, 0.09520151, 0.070176  , 0.06675569])